In [1]:
import numpy as np
import re
import spacy

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# from gensim.utils import simple_preprocess
from gensim import corpora
# from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, TfidfModel, ldamulticore

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



Using TensorFlow backend.
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/turambar/.local/lib/python3.6/site-packages/tensorflo

In [2]:
text_titles = ['Principles_of_morals_Hume', 'Political_discourses_Hume', 'Dialogues_natural_religion_Hume','Concerning_human_understanding_Hume',
'Cratylus_Plato', 'Apology_criton_phaedo_Plato', 'Gorgias_Plato', 'Republic_Plato',
 'History_peter_great_Voltaire', 'Socrates_Voltaire', 'Philosophical_dictionary_Voltaire', 'Candide_Voltaire',
 'Analysis_Mind_Russell', 'Mysticism_logic_Russell', 'Problems_philosophy_Russell', 'Roads_freedom_Russell', 
 'Pure_reason_Kant', 'Practical_reason_Kant', 'Judgment_Kant', 'Perpetual_peace_Kant']

authors_tag = ['hume', 'plato', 'voltaire', 'russell', 'kant']


text_array = []
label = []
for title in text_titles:
    text_array.append(open(f'data_txt/{title}.txt').read().lower())
    label.append(title.split('_')[-1])
# data_array = np.array([text_array, label])

In [3]:
characters = set()
for i in range(len(text_array)):
    for obj in set(text_array[i]):
        characters.add(obj)
print(characters)

{'’', '6', '_', 'j', '!', 'ô', 'h', 'm', 'é', 'θ', '.', '”', 'î', 'ì', 'ŭ', ':', 'q', 'r', '3', 'γ', 'β', 'ἀ', 'p', 'e', 'ο', 'φ', 'y', '°', 'w', 'ü', 'u', 'ω', '7', ' ', '&', 'è', '*', 'g', 'l', 'α', 'æ', '\n', 'δ', 'σ', 'œ', '[', '9', '“', 'n', 'η', '‘', '—', ']', 'ë', 'a', '|', '\t', 'ρ', '\u200b', 'ä', '/', 's', 'ö', 'π', 'ï', 'z', '§', 'i', '£', '>', '=', 'â', '8', 'k', 'ç', 'ε', 'd', '{', 'ê', 'b', 'ἄ', '+', "'", ')', '2', '(', 'λ', '4', '-', ',', '}', '1', 'μ', '?', 'o', 'x', ';', '"', '`', '0', 'û', '^', 'τ', 'κ', 'ή', 't', 'c', 'f', 'v', 'à', '5', 'ν', 'ι'}


In [4]:
mantain = ['î', 'α', 'z', '+', 'ŭ', '6', '‘', 'ü', 'ν', 'μ', 'w', 'æ', 't',
           'η', 'ἄ', 'è', 'd', 'i', 'τ', 'v', '5', 'h', 'u', '0', '”', '“',
           '{', 'g', 'k', '4', '£', 'ä', '!', 'b', 'σ', 'a', 'φ', '8', "'", 
           'ἀ', 'c', 'ε', 'à', 'o', 'l', '?', 'j', 'œ', '3', 'ô', 'ö', ',', 
           '>', 'δ', '2', 's', 'f', ';', ')', 'x', 'ê', 'ρ', 'κ', '(', '7', 
           'ο', '1', '}', '&', 'm', 'ή', ':', '-', 'y', 'û', 'ï', '’', 'β', 
           'ι', 'θ', 'ω', 'p', 'r', 'q', 'e', 'ë', '.', 'â', '"',
           '/', '`', 'γ', ']', 'π', '[', '9', 'é', 'n', 'ì', 'λ', '=']

replace = []
for char in characters:
    if char not in mantain:
        replace.append(char)

In [22]:
print(replace)

['_', '°', ' ', '*', '\n', '—', '|', '\t', '\u200b', '§', 'ç', '^']


In [5]:
texts_clean = []
for i in range(len(text_array)):
    text = text_array[i]
    text = re.sub(f'{replace}', ' ', text)
    texts_clean.append(re.sub(r'(\[.+?\])|(\{.+?\})', '', text))

In [6]:
texts_clean[0]

'an enquiry concerning the principles of morals  by david hume  section i. of the general principles of morals.    disputes with men  pertinaciously obstinate in their principles  are  of all others  the most irksome; except  perhaps  those with persons  entirely disingenuous  who really do not believe the opinions they defend  but engage in the controversy  from affectation  from a spirit of opposition  or from a desire of showing wit and ingenuity  superior to the rest of mankind. the same blind adherence to their own arguments is to be expected in both; the same contempt of their antagonists; and the same passionate vehemence  in inforcing sophistry and falsehood. and as reasoning is not the source  whence either disputant derives his tenets; it is in vain to expect  that any logic  which speaks not to the affections  will ever engage him to embrace sounder principles.  those who have denied the reality of moral distinctions  may be ranked among the disingenuous disputants; nor is i

In [7]:
texts_clean = [text for text in texts_clean]

In [8]:
def spacy_preprocessing(texts):

    nlp = spacy.load('en_core_web_sm')
    nlp.max_length = 10000000
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    texts_tokenized = []
    for text in texts_clean:
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
        texts_tokenized.append(tokens)
    return texts_tokenized

In [9]:
document = [spacy_preprocessing(text) for text in texts_clean[:1]]

In [15]:
print(document[0][0])

['enquiry', 'concern', 'principle', 'moral', ' ', 'david', 'hume', ' ', 'section', 'i.', 'general', 'principle', 'moral', '.', '   ', 'dispute', 'man', ' ', 'pertinaciously', 'obstinate', 'principle', ' ', ' ', ' ', 'irksome', ';', ' ', ' ', 'person', ' ', 'entirely', 'disingenuous', ' ', 'believe', 'opinion', 'defend', ' ', 'engage', 'controversy', ' ', 'affectation', ' ', 'spirit', 'opposition', ' ', 'desire', 'show', 'wit', 'ingenuity', ' ', 'superior', 'rest', 'mankind', '.', 'blind', 'adherence', 'argument', 'expect', ';', 'contempt', 'antagonist', ';', 'passionate', 'vehemence', ' ', 'inforce', 'sophistry', 'falsehood', '.', 'reasoning', 'source', ' ', 'disputant', 'derive', 'tenet', ';', 'vain', 'expect', ' ', 'logic', ' ', 'speak', 'affection', ' ', 'engage', 'embrace', 'sounder', 'principle', '.', ' ', 'deny', 'reality', 'moral', 'distinction', ' ', 'rank', 'disingenuous', 'disputant', ';', 'conceivable', ' ', 'human', 'creature', 'seriously', 'believe', ' ', 'character', 'act

In [18]:
chuchu = [word for text in document for word in text]

In [19]:
dictionary = corpora.Dictionary(chuchu)
print(dictionary)

Dictionary(20806 unique tokens: [' ', '  ', '   ', '    ', '     ']...)


In [29]:
corpus = [dictionary.doc2bow(word) for text in document for word in text]
print(corpus)

In [30]:
print(len(corpus))

3544


In [ ]:
import spacy
nlp = spacy.load("en")
texts = []
for document in documents:
    text = []
    doc = nlp(document)
    for w in doc:
        if not w.is_stop and not w.is_punct and not w.like_num:
            text.append(w.lemma_)
    texts.append(text)
    print(texts)